In [1]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')
from jobpostlib import (cu, datetime, duration, freq, hau, humanize, nu, scrfcu, slrcu, su, t0, time, wsu, speech_engine)
import os

Pretty printing has been turned OFF
======== Neo4j/4.4.7 ========
Utility libraries created in 5 seconds



----

In [2]:

wsu.save_indeed_email_to_file(verbose=True)

['powershell.exe', '-ExecutionPolicy', 'Bypass', '-Command', 'Start-Process "C:\\Program Files\\Notepad++\\notepad++.exe" -ArgumentList "C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\job-hunting\\data\\html\\indeed_email.html"']


In [3]:

try: driver = wsu.get_driver(verbose=True); speech_engine.say('Browser available'); speech_engine.runAndWait()
finally:
    viewjob_links_set = set()
    file_path = '../data/html/indeed_email.html'
    page_soup = wsu.get_page_soup(file_path)

    # Get viewjob links
    css_selector = 'a[href^="https://www.indeed.com/rc/clk/dl?jk="]'
    link_soups_list = page_soup.select(css_selector)
    for link_soup in link_soups_list: viewjob_links_set.add(link_soup['href'])

    # Get pagead links and convert them to viewjob links
    pagead_links_set = set()
    css_selector = 'a[href^="https://www.indeed.com/pagead/clk"]'
    link_soups_list = page_soup.select(css_selector)
    for link_soup in link_soups_list: pagead_links_set.add(link_soup['href'])
    for pagead_link in pagead_links_set:
        wsu.driver_get_url(driver, pagead_link, verbose=False)
        viewjob_links_set.add(driver.current_url)
    
    display(len(viewjob_links_set))

Getting the FireFox driver
platform.system() = Windows
os.name = nt
gecko_driver_path = GeckoDriverManager().install() = C:\Users\daveb\.wdm\drivers\geckodriver\win64\v0.35.0\geckodriver.exe


2

In [4]:

files_list = []; close_notices = True
for url_str in viewjob_links_set:
    file_node_dict, files_list = su.load_indeed_posting_url(viewjob_url=url_str, driver=driver, files_list=files_list, close_notices=close_notices, slowed_for_readability=False, verbose=False)
    close_notices = False
speech_str = f'Fileing {len(files_list)} out of {len(viewjob_links_set)} postings complete. Delete the email.'
print(speech_str)
speech_engine.say(speech_str)
speech_engine.runAndWait()

Fileing 2 out of 2 postings complete. Delete the email.


In [5]:

import re

t1 = time.time()
try: driver.close()
except Exception as e: print(f'{e.__class__.__name__} error: {str(e).strip()}')
cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    
    # Delete each of the previous siblings of the target_div that are <div> elements and prettify the HTML
    wsu.clean_job_posting(file_path)
    
    page_soup = wsu.get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    assert row_div_list, f'{file_name} is missing <div id="jobDescriptionText">'
    for target_div in row_div_list:
        child_strs_list = hau.get_navigable_children(target_div, [])
        assert child_strs_list, f'{file_name} is missing its child strings'
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
speech_str = f'Populating {len(files_list)} out of {len(viewjob_links_set)} postings completed in {duration_str}'; speech_engine.say(speech_str); speech_engine.runAndWait()
files_list

['16bef081ae79c206_BI_Analyst_BI_Engineer_Remote_Indeed_com.html', '37fcf3423fe11604_Sr_Data_Scientist_Healthcare_AI_Analytics_Remote_Indeed_com.html']